### Imports

In [25]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from pprint import pprint

from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsRegressor


from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

Nesta seccção iremos comparar o comportamento de diversos ML models no conjunto de dados, analisar o comportamento dos
algoritmos calculando métricas de erro apropriadas e usando métodos de estimação do erro adequado. Por último pretendemos apresentar o melhor modelo.

In [3]:
path="C:\\Users\\guilh\\OneDrive - Universidade de Aveiro\\Guilherme\\dataset.csv"
data=pd.read_csv(path)
print(data)

               Y  drug1 MolWt  drug1 HeavyAtomMolWt  drug1 ExactMolWt  \
0       7.693530     4.552044              4.441072          4.549836   
1       7.778053     4.552044              4.441072          4.549836   
2      -1.198505     4.552044              4.441072          4.549836   
3       2.595684     4.552044              4.441072          4.549836   
4      -5.139971     4.552044              4.441072          4.549836   
...          ...          ...                   ...               ...   
19861  10.223066     5.948467              5.475267          5.944298   
19862  20.551627     5.948467              5.475267          5.944298   
19863  12.190316     5.948467              5.475267          5.944298   
19864  23.438547     5.948467              5.475267          5.944298   
19865  12.223791     5.948467              5.475267          5.944298   

       drug1 BertzCT  drug1 TPSA  morgan drug1 15  morgan drug1 33  \
0          11.258666    2.190289              0.0    

In [5]:
y=data["Y"]
X = data.drop('Y', axis=1)
X

,drug1 MolWt,drug1 HeavyAtomMolWt,drug1 ExactMolWt,drug1 BertzCT,drug1 TPSA,morgan drug1 15,morgan drug1 33,morgan drug1 36,morgan drug1 64,morgan drug1 80,...,Parametro 8775,Parametro 8776,Parametro 8777,Parametro 8778,Parametro 8779,Parametro 8780,Parametro 8781,Parametro 8782,Parametro 8783,Parametro 8784
0,4.552044,4.441072,4.549836,11.258666,2.190289,0.0,0.0,0.0,0.0,0.0,...,0.933708,0.433194,-0.705877,-0.797751,0.494978,-0.239587,1.279730,0.691891,-0.982503,-0.364565
1,4.552044,4.441072,4.549836,11.258666,2.190289,0.0,0.0,0.0,0.0,0.0,...,0.453011,0.868781,-0.548668,1.191511,0.947887,0.393898,1.241226,1.139293,-1.070203,-0.425415
2,4.552044,4.441072,4.549836,11.258666,2.190289,0.0,0.0,0.0,0.0,0.0,...,-0.367088,-0.495932,-0.639091,-0.346006,-0.034777,-1.268040,-0.277364,-0.423137,-1.065120,-1.002490
3,4.552044,4.441072,4.549836,11.258666,2.190289,0.0,0.0,0.0,0.0,0.0,...,0.852838,-0.152449,-0.653551,0.369509,-0.358118,1.586969,1.275677,0.621267,-0.365272,-0.818869
4,4.552044,4.441072,4.549836,11.258666,2.190289,0.0,0.0,0.0,0.0,0.0,...,-1.854093,-2.383995,0.068724,-0.583244,-2.238896,-1.288220,-1.650908,-1.705543,-0.714340,0.635244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19861,5.948467,5.475267,5.944298,8.967280,1.585104,0.0,0.0,0.0,1.0,1.0,...,0.989373,0.598182,-0.708544,-0.494359,-0.540534,-0.977627,-0.762502,0.882945,-1.073682,0.481022
19862,5.948467,5.475267,5.944298,8.967280,1.585104,0.0,0.0,0.0,1.0,1.0,...,0.159920,0.157617,2.134676,2.314793,0.981662,-0.019900,0.278455,-0.374227,-0.073559,-0.217856
19863,5.948467,5.475267,5.944298,8.967280,1.585104,0.0,0.0,0.0,1.0,1.0,...,0.159920,0.157617,2.134676,2.314793,0.981662,-0.019900,0.278455,-0.374227,-0.073559,-0.217856
19864,5.948467,5.475267,5.944298,8.967280,1.585104,0.0,0.0,0.0,1.0,1.0,...,0.345755,0.172750,-0.672222,-0.729491,0.774517,0.535449,0.406174,-0.436593,1.207464,-1.522136


Divir o dataset em 70 % treino , 15 % validação e 15% teste

In [18]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


Divisão do dataset em treino, validação e teste.
Agora o que nos interessa é :

X_train, y_train;

X_val, y_val;

X_test,y_test.

In [29]:
y_val

1123     36.599969
6659      6.689280
14040    -1.964928
8455    -10.739940
9710      3.982901
           ...    
16637    24.221914
934      -8.532292
2404     -5.741128
17583     6.112842
878      -7.257895
Name: Y, Length: 2980, dtype: float64

# Modelos de ML
A nossa task, prever o nivel de sinergia entre duas drogas, o que se trata de um problema de regressão, logo só poderemos utilizar modelos que se baseiem em regressão.

## Logistical Regression

Um modelo de regressão logística modela a relação entre as variáveis independentes e a probabilidade de pertencer a uma determinada classe usando uma função logística. 

Aqui os hiperparâmetros estão definidos, mas podiamos fazer algo para determinar a melhro combinação.

Depois de cada modelo já feito com parâmetreos definidos teoricamente, tem outro exemplo com a exploração dos melhores hiperparâmetros. Para discutir a melhor abordagem

In [32]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor

models_to_try = [
    ('Linear Regression', LinearRegression()),
    ('KNN', KNeighborsRegressor()),
    ('Decision Tree', DecisionTreeRegressor()),
    ('Random Forest', RandomForestRegressor())
    #('SVM', SVR()),  # Support Vector Machine para regressão
    #('AdaBoost', AdaBoostRegressor())
]

results = []

for model_name, model in models_to_try:
    # Treinamento do modelo
    model.fit(X_train, y_train)
    
    # Avaliação do desempenho no conjunto de validação
    y_test_pred = model.predict(X_test)
    
    # Cálculo das métricas
    mse_val = mean_squared_error(y_test, y_test_pred)
    r2_val = r2_score(y_test, y_test_pred)
    mae_val = mean_absolute_error(y_test, y_test_pred)
    
    # Armazenar resultados em um dicionário
    model_results = {
        'Model': model_name,
        'Mean Squared Error': mse_val,
        'R-squared': r2_val,
        'Mean Absolute Error': mae_val,
    }
    
    # Adicionar os resultados à lista
    results.append(model_results)

    # Exibição dos resultados
    print(f'{model_name} - Mean Squared Error (Validation): {mse_val:.2f}')

# Converter a lista de resultados em um DataFrame do pandas (opcional)
results_df = pd.DataFrame(results)

# Encontrar o modelo com o menor MSE
best_model_df = results_df.loc[results_df['Mean Squared Error'].idxmin()]

# Exibir o melhor modelo
print(f'\nMelhor Modelo Escolhido:')
print(best_model_df)


Linear Regression - Mean Squared Error (Validation): 408.23
KNN - Mean Squared Error (Validation): 391.93
Decision Tree - Mean Squared Error (Validation): 566.49
Random Forest - Mean Squared Error (Validation): 251.76

Melhor Modelo Escolhido:
Model                  Random Forest
Mean Squared Error        251.759894
R-squared                   0.458749
Mean Absolute Error        10.177774
Name: 3, dtype: object


In [34]:
results

[{'Model': 'Linear Regression',
  'Mean Squared Error': 408.2329743671923,
  'R-squared': 0.12235183198607524,
  'Mean Absolute Error': 14.072671672404226},
 {'Model': 'KNN',
  'Mean Squared Error': 391.9258701801778,
  'R-squared': 0.1574099997823699,
  'Mean Absolute Error': 13.335788427795746},
 {'Model': 'Decision Tree',
  'Mean Squared Error': 566.4933787850647,
  'R-squared': -0.21788759679058023,
  'Mean Absolute Error': 15.047101666279918},
 {'Model': 'Random Forest',
  'Mean Squared Error': 251.75989428147605,
  'R-squared': 0.45874874429723667,
  'Mean Absolute Error': 10.177774430809903}]

# ensemble voting (a fazer)

In [ ]:
from sklearn.ensemble import VotingRegressor

# Escolha os modelos que deseja incluir no ensemble
model_linear = LinearRegression()
model_rf = RandomForestRegressor()
model_svm = SVR()

# Crie um ensemble utilizando a média das previsões dos modelos
ensemble_model = VotingRegressor(estimators=[
    ('Linear Regression', model_linear),
    ('Random Forest', model_rf),
    ('SVM', model_svm)
])

# Treine o ensemble
ensemble_model.fit(X_train, y_train)

# Faça previsões usando o ensemble
y_pred_ensemble = ensemble_model.predict(X_test)

# Avalie o desempenho do ensemble
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)
mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)

# Exiba os resultados do ensemble
print(f'Ensemble - Mean Squared Error (Validation): {mse_ensemble:.2f}')
print(f'Ensemble - R-squared (Validation): {r2_ensemble:.2f}')
print(f'Ensemble - Mean Absolute Error (Validation): {mae_ensemble:.2f}')


# ensemble stacking (a fazer)

# daqui pra baixo ainda nao é para fazer!

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
# Definir os hiperparâmetros 
parameters = {
    'penalty': ['l1', 'l2', 'none'],
    'C': [0.1, 1, 10],
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'max_iter': [10,100, 200]
}

# Criar o modelo de regressão logística
logistic_regression = LogisticRegression()

# Criar o objeto GridSearchCV
grid_search = GridSearchCV(logistic_regression, parameters, scoring='f1', cv=5)

# Executar a busca em grid para encontrar os melhores hiperparâmetros
grid_search.fit(train_X[:, 3:], train_y)  #FUI USANDO SEMPRE [:, 3:] (todas as linhas e descartando as 3 primeiras colunas) MAS NÃO ESTOU A VER NO DATASET QUAIS COLUNAS INCLUIR, POR ISSO QUANDO SE TIVER O DATASET TEM QUE SE ADAPTAR ISTO EM TODOS ABAIXO

# Obter os melhores hiperparâmetros encontrados
best_params_LT = grid_search.best_params_
best_params_LT

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
#hiperparametros
parameters = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

dt_classifier = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt_classifier, parameters, scoring='f1', cv=5)
grid_search.fit(X_tr[:, 3:], train_y)

best_params_DT = grid_search.best_params_
best_params_DT

#### Com exploração de hiperparâmetros para determinar a melhor combinação

## SVR (Support Vector Regression)

O SVR é uma técnica de ML para regressão, baseada em Support Vector Machines (SVM), que procura prever valores contínuos ao otimizar uma função de margem de erro entre as previsões do modelo e os valores reais dos dados de treino

In [ ]:
svr = SVR()
svr.fit(train_X, train_y)

print(f'Accuracy: {svr.score(test_X, test_y):.2%}')

pred_y = svr.predict(test_X)
pprint(classification_report(test_y, pred_y))

#### Com exploração de hiperparâmetros para determinar a melhor combinação

In [ ]:
# Definir os parâmetros a ajustar para SVR
parameters_svr = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

# Criar o regressor SVR
SVR_regressor = SVR()

# Criar o objeto GridSearchCV
grid_search_svr = GridSearchCV(SVR_regressor, parameters_svr, scoring='neg_mean_squared_error', cv=5)

# Executar o grid search usando os dados de treino (X_tr e y_tr)
grid_search.fit(train_X[:, 3:], train_y)

# Obter os melhores parâmetros
best_params_SVR = grid_search.best_params_
best_params_SVR


In [ ]:
#TREINO

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Supondo que você tenha um conjunto de dados train_X, train_y
# e que 'best_params_SVM' contenha os melhores hiperparâmetros obtidos

# Criar um objeto MinMaxScaler
min_max_scaler = MinMaxScaler()

# Definir os parâmetros a ajustar para SVM
parameters_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'degree': [2, 3, 4],
}

# Criar o classificador SVM
SVM_classifier = SVC()

# Criar o objeto GridSearchCV
grid_search_svm = GridSearchCV(SVM_classifier, parameters_svm, scoring='accuracy', cv=5)

# Dividir os dados usando StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

for train_index, test_index in sss.split(train_X, train_y):
    X_SVM_train, X_SVM_test, y_SVM_train, y_SVM_test = train_X[train_index, 3:], train_X[test_index, 3:], train_y[train_index], train_y[test_index]
    
    # Normalizar os dados
    X_SVM_train_new = min_max_scaler.fit_transform(X_SVM_train)
    X_SVM_test_new = min_max_scaler.transform(X_SVM_test)
    
    # Executar o grid search usando os dados de treino
    grid_search_svm.fit(X_SVM_train_new, y_SVM_train)

    # Obter os melhores parâmetros
    best_params_SVM = grid_search_svm.best_params_
    
    # Criar o modelo SVM com os melhores hiperparâmetros
    SVM_classifier = SVC(**best_params_SVM)
    
    # Treinar o modelo
    SVM_classifier.fit(X_SVM_train_new, y_SVM_train)


In [ ]:
#Avaliação
X_new = train_X[:,3:]
X_new = min_max_scaler.fit_transform(X_new)
KN_classifier.fit(X_new, y_tr)
y_tr_predict = KN_classifier.predict(X_new)
print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))

tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

In [ ]:
#TESTE

X_te_new = test_X[:,3:]

X_te_new = min_max_scaler.transform(X_te_new)

y_SVM_pred = KN_classifier.predict(X_te_new)

print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_SVM_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_SVM_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_SVM_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn + fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp + fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print('Accuracy on Test set: ', accuracy)
precision=tp/(tp+fp)
print("Precision on Test set: ", precision)

In [ ]:
#Confusion Matrix

# Calcula a matriz de confusão
confusion = confusion_matrix(y_te, y_SVM_pred)

# Obtém os valores dos verdadeiros positivos (tp), verdadeiros negativos (tn),
# falsos positivos (fp) e falsos negativos (fn)
tn, fp, fn, tp = confusion.ravel()

# Imprime os valores da matriz de confusão
print("Confusion Matrix:")
print(confusion)

# Plota a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(confusion, annot=True, cmap='Greens', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Model Validation

In [ ]:
# Model validation

# cross validation
scores = cross_val_score(logreg, train_X, train_y, cv=5)
print('Cross validation scores:', scores)
print('Mean cross validation score:', scores.mean())

In [ ]:
# Bootstrap
scores = []
for i in range(1000):
    X_boot, y_boot = resample(X_train, y_train)
    logreg.fit(X_boot, y_boot)
    scores.append(logreg.score(X_test, y_test))
    
print('Mean bootstrap score:', np.mean(scores))
print('Standard deviation of bootstrap scores:', np.std(scores))

## Hyperparameter tuning

We will use random search to find the best hyperparameters for our models.

In [ ]:
# Hyperparameter tuning

# random forest hyperparameter tuning
param_grid = {'n_estimators': [10, 100, 1000],
              'max_depth': [None, 5, 10, 20],
              'max_features': ['auto', 'sqrt'],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}

rf = RandomForestRegressor(random_state=42)
rand_search = RandomizedSearchCV(rf, param_grid, cv=5, verbose=2, n_jobs=-1, n_iter=5)
rand_search.fit(train_X, train_y)
rand_search.best_params_, rand_search.best_score_, rand_search.best_estimator_.score(test_X, test_y)
mse = mean_squared_error(test_X, pred_y)
print(f'Mean Squared Error: {mse}')


#grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
#grid_search.best_estimator_ # best model

## Save and load the best model

In [ ]:
# save the best model

joblib.dump(rand_search.best_estimator_, 'best_model.pkl')

# load the best model
best_model = joblib.load('best_model.pkl')
best_model.score(test_X, test_y)

## Model interpretation

scikit-learn provides multiple methods for model interpretation. Here we will see feature importance and permutation importance.


In [ ]:
# Model interpretation
# feature importance

rf = RandomForestRegressor(random_state=42, n_estimators=1000, max_depth=10, max_features='sqrt', min_samples_split=5, min_samples_leaf=2)
rf.fit(train_X, train_y)
rf.feature_importances_

# plot feature importance
importances = pd.Series(rf.feature_importances_, index=selected_columns)  #este selected columns vão ser as selecionadas na feature selection, mas como não está importado não lê esta variável
importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
# feature importance based on permutation importance

perm_importance = permutation_importance(rf, test_X, test_y)
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(selected_columns[sorted_idx[:10]], perm_importance.importances_mean[sorted_idx[:10]])
plt.xlabel("Permutation Importance")
plt.show()

In [ ]:

pos_neg_idx = np.concatenate((sorted_idx[:10], sorted_idx[-10:]))
plt.barh(selected_columns[pos_neg_idx], perm_importance.importances_mean[pos_neg_idx])
plt.xlabel("Permutation Importance")
plt.show()